# Animations with Earth Engine
**Set the environment**

Initialize Earth Engine account

In [1]:
from ee_satellite_imagery import Composite, Animation
from ipywidgets import interact
import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint
import getpass
import requests
import json
import os
import ee
ee.Initialize()

## Satellite imagery
**Area of Interest**

In [2]:
geometry = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              17.60009765625,
              -14.744671057048434
            ],
            [
              17.77484893798828,
              -14.744671057048434
            ],
            [
              17.77484893798828,
              -14.576273583944515
            ],
            [
              17.60009765625,
              -14.576273583944515
            ],
            [
              17.60009765625,
              -14.744671057048434
            ]
          ]
        ]
      }
    }
  ]
}

geometry = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              17.64181137084961,
              -14.670037466743915
            ],
            [
              17.73957252502441,
              -14.670037466743915
            ],
            [
              17.73957252502441,
              -14.638565836897298
            ],
            [
              17.64181137084961,
              -14.638565836897298
            ],
            [
              17.64181137084961,
              -14.670037466743915
            ]
          ]
        ]
      }
    }
  ]
}

### Static image

In [16]:
composite = Composite(geometry=geometry, startDate = ee.Date('2019-01-01'), stopDate = ee.Date('2019-12-31'), collection = 'Sentinel2')
composite.read_composite()
composite.display_composite_folium()

**Save image as `png`**

In [17]:
composite.save_composite_as_png('../data/', 'Sentinel2_6')

### Time-lapse

In [9]:
animation = Animation(geometry=geometry, start_year = 1988, stop_year = 2019, instrument = 'Landsat')

**Numpy array**

In [10]:
video = animation.video_as_array(dimensions=1024)
video = animation.video_add_elements(logo_path='../data/mongabay-horizontal.jpg', y_pixels=20)

Image number: 0
Image number: 1
Image number: 2
Image number: 3
Image number: 4
Image number: 5
Image number: 6
Image number: 7
Image number: 8
Image number: 9
Image number: 10
Image number: 11
Image number: 12
Image number: 13
Image number: 14
Image number: 15
Image number: 16
Image number: 17
Image number: 18
Image number: 19
Image number: 20
Image number: 21
Image number: 22
Image number: 23
Image number: 24
Image number: 25
Image number: 26
Image number: 27
Image number: 28
Image number: 29
Image number: 30
Image number: 31


In [6]:
@interact(frame=(0, video.shape[0]-1))
def show_frame(frame=0):
    plt.figure(figsize=(20,10))
    plt.imshow(video[frame,:,:,:])

interactive(children=(IntSlider(value=0, description='frame', max=3), Output()), _dom_classes=('widget-interac…

**Create animations from a Numpy Array**

In [11]:
animation.create_movie_from_array('../data/ladsat8.mp4', output_format='mp4')

**Upload animation to Google cloud storage**

In [ ]:
source_file_name = '../data/movie.mp4'
destination_blob_name = 'movie-tiles/mongabay/movie.mp4'
animation.upload_blob(source_file_name, destination_blob_name)

**Remove file from local directory**

In [ ]:
os.remove('../data/movie.mp4')

**Display animation on map**

In [ ]:
url = "https://storage.cloud.google.com/skydipper_materials/movie-tiles/mongabay/movie.mp4"
animation.display_animation(url)

## Deploying Cloud Functions

To create a Google Cloud Function we need a [Google Cloud Project](https://cloud.google.com/resource-manager/docs/creating-managing-projects) and [gcloud SDK](https://cloud.google.com/sdk/docs/).

If we have already some projects we can check them by typing:

`gcloud projects list`
```
PROJECT_ID        NAME                     PROJECT_NUMBER
gef-ld-toolbox    gef-ld-toolbox           1080184168142
gfw-apis          Global Forest Watch API  872868960419
resource-watch    Resource Watch           312603932249
skydipper-196010  skydipper                230510979472
soc-platform      SOC Platform             345072612231
```
and select one by:

`gcloud config set project skydipper-196010`

Then create a function by creating a `main.py` file with some python code on it:

```python
from ee_satellite_imagery import Animation
import json
import ee

service_account = 'skydipper@skydipper-196010.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'privatekey.json')
ee.Initialize(credentials)

def serializer(url):

    return {
        'download_url': url
    }

def animation(request):
    request = request.get_json()
    
    animation = Animation(geometry=request['geometry'], start_year = request['start_year'], stop_year = request['stop_year'], instrument = request['instrument'])
    
    # Numpy array
    video = animation.video_as_array(dimensions=request['dimensions'])

    # Create animations from a Numpy Array
    animation.create_movie_from_array('movie.mp4', output_format='mp4')
    
    # Upload animation to Google cloud storage
    print('Uploadong animation to Google cloud storage')
    source_file_name = 'movie.mp4'
    destination_blob_name = 'movie-tiles/mongabay/movie.mp4'
    animation.upload_blob(source_file_name, destination_blob_name)

    url = "https://storage.cloud.google.com/skydipper_materials/movie-tiles/mongabay/movie.mp4"

    return json.dumps(serialize_response(url))
```

In the same directory include the `privatekey.json` with the [service account keys](https://cloud.google.com/iam/docs/creating-managing-service-account-keys) and the `requirements.txt` file. 

Finally cd to that directory and deploy the cloud Function with the following command:

`gcloud beta functions deploy animation --runtime python37 --trigger-http --timeout 540` 

Note that the cloud function name matches the name of the function we defined in code: `animation`.

## Call the Cloud Functions

You can invoke Cloud Functions with an HTTP request using the POST, PUT, GET, DELETE, and OPTIONS HTTP methods. To find the `URL` that triggers the function open the details of the created function and click on the `Trigger` tab.

Get your Google OAuth token with the following command:

`gcloud auth print-identity-token`

In [ ]:
token = getpass.getpass('Provide a Google OAuth token here:')

In [ ]:
payload =   {
    "start_year": 2010,
    "stop_year": 2019,
    "instrument": "Landsat",
    "dimensions":512,
    "geometry": {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              17.60009765625,
              -14.744671057048434
            ],
            [
              17.77484893798828,
              -14.744671057048434
            ],
            [
              17.77484893798828,
              -14.576273583944515
            ],
            [
              17.60009765625,
              -14.576273583944515
            ],
            [
              17.60009765625,
              -14.744671057048434
            ]
          ]
        ]
      }
    }
  ]
}
}

In [ ]:
%%time
url = f'https://us-central1-skydipper-196010.cloudfunctions.net/animation'

headers = {'Authorization': 'Bearer ' + token, 'Content-Type': 'application/json'}
#headers = {'Content-Type': 'application/json'}

r = requests.post(url, data=json.dumps(payload), headers=headers)
pprint(r.json())